w1 - PR

w2 - LAR 

w3 - OP_Price 

w4 - AVPI 

In [ ]:
import os
import pandas as pd
import numpy as np
from itertools import product

In [ ]:
# Function to generate all valid weight combinations within specified ranges
def generate_weight_combinations():
    """
    Generate all valid weight combinations with specific constraints:
    - Weights are within the specified ranges for each parameter.
    - Weights are ordered: w1 > w2 > w3 > w4.
    - Sum of weights is 1.
    """
    step = 0.01  
    valid_combinations = []

    w1_range = np.linspace(0.23, 0.51, int((0.51 - 0.23) / step) + 1)
    w2_range = np.linspace(0.19, 0.47, int((0.47 - 0.19) / step) + 1)
    w3_range = np.linspace(0.15, 0.43, int((0.43 - 0.15) / step) + 1)
    w4_range = np.linspace(0.00, 0.15, int((0.15 - 0.00) / step) + 1)

    for w1, w2, w3, w4 in product(w1_range, w2_range, w3_range, w4_range):
        if (w1 > w2 > w3 > w4 and 
            abs(w1 + w2 + w3 + w4 - 1) < step   # Ensure sum is approximately 1
        ):  
            valid_combinations.append((round(w1, 2), round(w2, 2), round(w3, 2), round(w4, 2)))

    return valid_combinations


In [ ]:
# Function to calculate Adjstment Factor (F)
def calculate_videal(df, weights):
    """
    Calculate Adjstment Factor (F) based on the given weights.
    
    Parameters:
    - df: Input DataFrame containing relevant columns.
    - weights: Tuple of weights (w1, w2, w3, w4).
    
    Returns:
    - DataFrame with updated 'F' column.
    """
    w1, w2, w3, w4 = weights

    df['F'] = round(
        (1 / (1 + (w1 * df['PR_Scaled']))) *
        (1 + (w2 * df['LAR_Scaled'])) * 
        (1 / (1 + (w4 * df['OP_Price_Scaled']))) *
        (1 + (w3 * df['AVPI_Scaled'])), 6)

    return df


In [ ]:
# Main function to generate weight combinations and save results
def main(input_file_path, output_folder):
    """
    Main function to process the input file, generate weight combinations, and save results.
    
    Parameters:
    - input_file_path: Path to the input CSV file.
    - output_folder: Folder to save the results for each weight combination.
    """
    os.makedirs(output_folder, exist_ok=True)

    df = pd.read_csv(input_file_path)

    weight_combinations = generate_weight_combinations()
    print(f"Generated {len(weight_combinations)} valid weight combinations.")

    for weights in weight_combinations:

        temp_df = calculate_videal(df.copy(), weights)

        temp_df['weights'] = f"w1={weights[0]}, w2={weights[1]}, w3={weights[2]}, w4={weights[3]}"

        output_filename = f"results_w1_{weights[0]}_w2_{weights[1]}_w3_{weights[2]}_w4_{weights[3]}.csv"
        output_path = os.path.join(output_folder, output_filename)

        temp_df[['Date', 'F', 'weights', 'Votable Supply']].to_csv(output_path, index=False)
        print(f"Saved: {output_path}")


In [ ]:
# Input and output paths
input_file_path = "../../Dataset/Ideal_Votable_Supply_Data/All_Parameters_Data.csv" 
output_folder = "../../Dataset/Ideal_Votable_Supply_Data/All_Weight_Combinations"  

# Main function
if __name__ == "__main__":
    main(input_file_path, output_folder)